In [1]:
from pyagnps import climate

from pathlib import Path

import pynldas2

import hvplot.pandas #no.qa

In [2]:
clm_annagnps = climate.ClimateAnnAGNPSCoords(coords=(-89.1, 34.1), start="1980-01-01", end="1981-07-31", date_mode="local")

In [3]:
root_dir = Path('../../outputs/climate/')

df = clm_annagnps.query_nldas2_generate_annagnps_climate_daily(output_filepath= root_dir / "climate_daily.csv", float_format='%.2f')

In [4]:
df

,Month,Day,Year,Max_Air_Temperature,Min_Air_Temperature,Precip,Dew_Point,Sky_Cover,Wind_Speed,Wind_Direction,Solar_Radiation,Storm_Type_ID,Potential_ET,Actual_ET,Actual_EI,Input_Units_Code
date,,,,,,,,,,,,,,,,
1980-01-01,1,1,1980,9.200000,2.830000,0.0000,0.702799,None,3.603718,114.973831,122.838333,None,2.3140,None,None,1
1980-01-02,1,2,1980,10.050000,1.010000,0.3343,-3.407632,None,0.983013,151.945419,84.589165,None,1.3207,None,None,1
1980-01-03,1,3,1980,7.480000,2.440000,10.3941,-1.411954,None,1.936419,273.504150,41.893166,None,0.6817,None,None,1
1980-01-04,1,4,1980,6.410000,1.340000,2.3033,0.362525,None,5.365164,137.888397,107.222664,None,2.5982,None,None,1
1980-01-05,1,5,1980,4.740000,0.250000,0.0000,-1.992065,None,3.231326,155.017639,92.897667,None,2.1424,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981-07-27,7,27,1981,32.840000,23.959999,4.7383,21.775084,None,3.206521,123.006256,261.952423,None,6.7906,None,None,1
1981-07-28,7,28,1981,31.459999,25.770000,0.0198,22.449381,None,3.854196,55.153370,223.675995,None,6.3431,None,None,1
1981-07-29,7,29,1981,30.299999,23.690001,0.0000,22.216511,None,2.245382,148.087234,239.437332,None,6.1820,None,None,1


In [5]:
df_station = clm_annagnps.generate_climate_station_file(output_filepath= root_dir / 'climate_station.csv',
                                                        elevation='3dep')

In [6]:
df_station

,Version,Input Units Code,Climate Station Name,Beginning Climate Date 'mm/dd/yyyy',Ending Climate Date 'mm/dd/yyyy',Latitude,Longitude,Elevation,Temperature Lapse Rate,Precipitation N,...,2nd Elevation Rain Factor,2 Yr 24 hr Precipitation,Calibration or Areal Correction Coefficient,Calibration or Areal Correction Exponent,Minimum Interception Evaporation,Maximum Interception Evaporation,Winter Storm Type ID,Spring Storm Type ID,Summer Storm Type ID,Autumn Storm Type ID
0,6.0,1,Station,01/01/1980,07/31/1981,34.1,-89.1,102.48481,,,...,,,,,,,,,,


### Dashboard

In [7]:
import hvplot.pandas
import hvplot

import holoviews as hv
import panel as pn
from panel.widgets import Select, MultiSelect, MultiChoice

columns = ['Max_Air_Temperature', 'Min_Air_Temperature', 'Precip', 'Dew_Point', 'Wind_Speed', 'Wind_Direction', 'Solar_Radiation', 'Potential_ET']

# dropdown = Select(options=columns, sizing_mode='fixed', height=30)
# def plot_timeseries(df, column):
#     return hv.Curve((df.index, df[column])).opts(responsive=True, height=600)

# interactive_plot = pn.bind(plot_timeseries, df, dropdown)

dropdown = MultiChoice(options=columns)
def plot_multi_timeseries(df, columns):
    if columns:
        overlay_list = []
        for col in columns:
            overlay_list.append(hv.Curve((df.index, df[col]))*hv.Scatter((df.index, df[col])))
        return hv.Overlay(overlay_list).opts(responsive=True, height=800)

interactive_plot = pn.bind(plot_multi_timeseries, df, dropdown)

app = pn.Column(dropdown, interactive_plot, sizing_mode="stretch_width")
app.show(threaded=True)
# app.height = 1000
# app

# app.save("climate.html", embed=True)


<StoppableThread(Thread-25 (get_server), started 34348)>

Launching server at http://localhost:60317


c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


In [16]:
app.show()

Launching server at http://localhost:54367


c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a v

In [7]:
import py3dep
elev_airmap = py3dep.elevation_bycoords((-89.1, 34.1), source='airmap')
elev_tep = py3dep.elevation_bycoords((-89.1, 34.1), source='tep')
elev_tnm = py3dep.elevation_bycoords((-89.1, 34.1), source='tnm')

In [8]:
print(f"Airmap: {elev_airmap}, TEP: {elev_tep}, TNM: {elev_tnm}")

Airmap: 101, TEP: 102.48480987548828, TNM: 102.227973938
